# Landscape Evolution Simulator - Example Workflow

This notebook demonstrates the complete workflow of the refactored landscape evolution simulator.

## Overview

The simulator follows this conceptual architecture:

1. **World State**: Surface elevation, layer interfaces, material properties, mobile sediment
2. **External Forcing**: Tectonic uplift/subsidence, climate/weather patterns
3. **Water Routing**: Flow directions, flow accumulation, drainage networks
4. **Geomorphic Processes**: Channel erosion, hillslope diffusion, weathering
5. **Layer-Aware Stratigraphy**: Erosion removes from top layers, deposition adds material
6. **Time-Stepping**: Integrate all processes over many time steps

## Example 1: Simple Demonstration

First, let's create a simple simulation with default parameters to understand the framework.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import the landscape evolution framework
from landscape_evolution import (
    WorldState,
    create_simple_simulator,
    FlowRouter
)
from landscape_evolution.terrain_generation import (
    quantum_seeded_topography,
    denormalize_elevation
)
from landscape_evolution.initial_stratigraphy import (
    create_slope_dependent_stratigraphy
)
from landscape_evolution.visualization import (
    plot_initial_vs_final,
    plot_erosion_deposition_maps,
    plot_river_network,
    plot_evolution_summary,
    plot_cross_section
)

print("Imports successful!")

### Step 1: Generate Initial Terrain

We'll use your existing terrain generation code to create the initial topography.

In [ ]:
# Parameters
N = 256  # Grid size (keep smaller for faster computation)
pixel_scale_m = 100.0  # 100 m per pixel
elev_range_m = (0.0, 1000.0)  # Elevation range

# Generate terrain using your existing code
z_norm, rng = quantum_seeded_topography(
    N=N,
    beta=3.1,
    warp_amp=0.12,
    ridged_alpha=0.18,
    random_seed=42  # For reproducibility
)

# Convert to actual elevation
surface_elev = denormalize_elevation(z_norm, elev_range_m)

# Visualize
plt.figure(figsize=(10, 8))
plt.imshow(surface_elev, cmap='terrain', origin='lower')
plt.colorbar(label='Elevation (m)')
plt.title('Initial Terrain')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.show()

print(f"Terrain generated: {N}×{N} grid")
print(f"Elevation range: [{surface_elev.min():.1f}, {surface_elev.max():.1f}] m")

### Step 2: Initialize World State with Stratigraphy

Create a `WorldState` and initialize it with simple stratigraphy.

In [ ]:
# Define layer names (from top to bottom)
layer_names = [
    "Topsoil",
    "Colluvium",
    "Saprolite",
    "WeatheredBR",
    "Sandstone",
    "Shale",
    "Basement"
]

# Create world state
world = WorldState(
    nx=N,
    ny=N,
    pixel_scale_m=pixel_scale_m,
    layer_names=layer_names
)

# Initialize with slope-dependent stratigraphy
create_slope_dependent_stratigraphy(
    world,
    surface_elev=surface_elev,
    pixel_scale_m=pixel_scale_m,
    base_regolith_m=2.0,
    base_saprolite_m=5.0,
    bedrock_thickness_m=100.0
)

print("\nWorld State Initialized:")
print(world.summary())

### Step 3: Set Up External Forcing

Define tectonic uplift and climate conditions.

In [ ]:
from landscape_evolution import TectonicUplift, WeatherGenerator

# Tectonic uplift
tectonics = TectonicUplift(N, N, pixel_scale_m)
tectonics.set_uniform_uplift(1e-3)  # 1 mm/yr uniform uplift

# Weather/climate
weather = WeatherGenerator(
    N, N, pixel_scale_m,
    mean_annual_precip_m=1.0,  # 1 m/yr rainfall
    wind_direction_deg=270.0,  # Wind from west
    orographic_factor=0.5
)

print("External forcing configured:")
print(f"  {tectonics}")
print(f"  {weather}")

### Step 4: Create and Run Simulator

Now we create the simulator and run it for a specified duration.

In [ ]:
from landscape_evolution import LandscapeEvolutionSimulator

# Create simulator
simulator = LandscapeEvolutionSimulator(
    world=world,
    tectonics=tectonics,
    weather=weather,
    snapshot_interval=50,  # Save snapshot every 50 steps
    verbose=True
)

# Run simulation
total_time = 10000.0  # 10,000 years
dt = 10.0  # 10-year time steps

print(f"\nRunning simulation for {total_time} years with dt={dt} years...")
history = simulator.run(total_time=total_time, dt=dt)

print(f"\nSimulation complete!")
print(f"Number of snapshots: {len(history.times)}")

### Step 5: Visualize Results

Now let's visualize the evolution.

In [ ]:
# Plot initial vs final topography
plot_initial_vs_final(history, pixel_scale_m)

In [ ]:
# Plot erosion and deposition maps
plot_erosion_deposition_maps(history, pixel_scale_m)

In [ ]:
# Compute and plot river network
flow_router = FlowRouter(pixel_scale_m)
flow_dir, slope, flow_accum = flow_router.compute_flow(world.surface_elev)

plot_river_network(
    world.surface_elev,
    flow_accum,
    pixel_scale_m,
    threshold_cells=0.01 * N * N  # 1% of domain
)

In [ ]:
# Plot cross-section
plot_cross_section(world, row=N//2, vertical_exaggeration=2.0)

In [ ]:
# Comprehensive summary
plot_evolution_summary(history, world, flow_accum)

## Example 2: Using Your Original Stratigraphy Code

To use your sophisticated `generate_stratigraphy()` function from `Project.ipynb`:

```python
# 1. Generate terrain and stratigraphy with your original code
from Project import generate_stratigraphy  # Import from your notebook

z_norm, rng = quantum_seeded_topography(N=512, random_seed=42)
strata = generate_stratigraphy(
    z_norm, rng,
    elev_range_m=700.0,
    pixel_scale_m=10.0,
    # ... all your other parameters ...
)

# 2. Create WorldState
layer_names = list(strata['thickness'].keys())
world = WorldState(512, 512, pixel_scale_m=10.0, layer_names=layer_names)

# 3. Initialize from generated stratigraphy
from landscape_evolution.initial_stratigraphy import initialize_world_from_stratigraphy

initialize_world_from_stratigraphy(
    world,
    surface_elev=strata['surface_elev'],
    thickness=strata['thickness']
)

# 4. Now you can evolve the landscape!
simulator = LandscapeEvolutionSimulator(world, tectonics, weather)
history = simulator.run(total_time=10000.0, dt=10.0)
```

This preserves all your detailed layer generation logic while allowing you to evolve the landscape over time.

## Example 3: Custom Forcing Patterns

You can create more complex forcing patterns:

In [ ]:
# Regional uplift pattern (high in center, low at edges)
tectonics_regional = TectonicUplift(N, N, pixel_scale_m)
tectonics_regional.set_regional_pattern(
    center_rate=2e-3,  # 2 mm/yr in center
    edge_rate=0.0,     # No uplift at edges
    center_frac=0.5    # Center is inner 50% of domain
)

# Visualize uplift pattern
plt.figure(figsize=(10, 8))
plt.imshow(tectonics_regional.uplift_rate * 1000, cmap='RdBu_r', origin='lower')
plt.colorbar(label='Uplift rate (mm/yr)')
plt.title('Regional Uplift Pattern')
plt.show()

## Example 4: Analyzing Different Time Steps

The `SimulationHistory` object stores snapshots that you can analyze:

In [ ]:
# Extract specific snapshots
times = history.times
surfaces = history.surface_snapshots

# Compare multiple time steps
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
indices = [0, len(surfaces)//4, len(surfaces)//2, 3*len(surfaces)//4, len(surfaces)-1]

for idx, ax_idx in zip(indices[:5], [(0,0), (0,1), (0,2), (1,0), (1,1)]):
    ax = axes[ax_idx]
    ax.imshow(surfaces[idx], cmap='terrain', origin='lower')
    ax.set_title(f't = {times[idx]:.0f} years')
    ax.axis('off')

axes[1, 2].axis('off')
plt.tight_layout()
plt.show()

## Key Concepts Demonstrated

### 1. Clear State of the World

The `WorldState` object maintains:
- Surface elevation at each (x, y)
- Layer interfaces for each geological layer
- Material properties (erodibility, density, permeability, weathering rate)
- Mobile sediment thickness

### 2. External Forcing

- **Tectonic**: Uniform or spatially-varying uplift/subsidence
- **Climate**: Rainfall with orographic enhancement, wind direction

### 3. Water Routing

- D8 flow direction algorithm
- Flow accumulation (drainage area)
- Channel network identification

### 4. Geomorphic Processes

- **Channel erosion**: Stream power law (E = K × A^m × S^n)
- **Hillslope diffusion**: Smooths slopes (∂z/∂t = κ ∇²z)
- **Weathering**: Converts bedrock to mobile regolith
- **Sediment transport**: Moves and deposits sediment

### 5. Layer-Aware Stratigraphy

- Erosion removes material from topmost layers first
- Deposition adds material to surface layers
- Layer ordering is always enforced
- No layer inversions allowed

### 6. Time-Stepping Loop

Each time step:
1. Apply tectonic uplift
2. Generate rainfall
3. Route water
4. Compute erosion and deposition
5. Update stratigraphy
6. Enforce constraints

## Next Steps

- Integrate your full `generate_stratigraphy()` function
- Adjust process parameters for your specific application
- Add more sophisticated forcing patterns
- Implement structural geometry (dip, folds)
- Extend to longer time scales or finer resolutions